# Strategies in Federated Learning

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/tutorial/Flower-1-Intro-to-FL-PyTorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> Join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌻 We'd love to hear from you in the `#introductions` channel! If anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower Strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [1]:
# %pip install -q flwr[simulation] torch torchvision

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 1.13.1+cu117 and Flower 1.3.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [6]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-02-27 15:47:42,851 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-02-27 15:47:58,512	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
E0227 15:48:09.053167700   14900 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509289.052203500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
INFO flwr 2023-02-27 15:48:09,445 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:10.246.68.42': 1.0, 'memory': 734097408.0, 'object_store_memory': 367048704.0, 'CPU': 8.0}
INFO flwr 2023-02-27 15:48:09,456 | server.py:86 | Initializing global parameters
INFO flwr 2023-02-27 15:48:09,497 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-02-27 15:48:09

(launch_and_fit pid=15343) [Client 8] fit, config: {}


(raylet) E0227 15:48:34.826798100   15990 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509314.826763900","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}


(launch_and_fit pid=15343) Epoch 1: train loss 0.06502389162778854, accuracy 0.22955555555555557
(launch_and_fit pid=15343) [Client 6] fit, config: {}
(launch_and_fit pid=15345) [Client 0] fit, config: {}
(launch_and_fit pid=15345) Epoch 1: train loss 0.0649348720908165, accuracy 0.2371111111111111
(launch_and_fit pid=15345) [Client 6] fit, config: {}


DEBUG flwr 2023-02-27 15:48:54,495 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-02-27 15:48:54,516 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-02-27 15:48:54,519 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


...

DEBUG flwr 2023-02-27 15:50:11,544 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-02-27 15:50:11,564 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=15346) Epoch 1: train loss 0.05534721538424492, accuracy 0.3526666666666667
(launch_and_evaluate pid=15346) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=15346) [Client 1] evaluate, config: {}


DEBUG flwr 2023-02-27 15:50:21,974 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-02-27 15:50:21,975 | server.py:144 | FL finished in 132.40289959999973
INFO flwr 2023-02-27 15:50:21,977 | app.py:202 | app_fit: losses_distributed [(1, 0.06310507273674011), (2, 0.05618654195467632), (3, 0.05283888673782348)]
INFO flwr 2023-02-27 15:50:21,978 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-02-27 15:50:21,979 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-02-27 15:50:21,981 | app.py:205 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=15346) [Client 5] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06310507273674011
	round 2: 0.05618654195467632
	round 3: 0.05283888673782348

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [7]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-02-27 15:50:22,257 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-02-27 15:50:37,970	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-02-27 15:50:44,312 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1141410201.0, 'memory': 2282820404.0, 'CPU': 8.0, 'node:10.246.68.42': 1.0}
INFO flwr 2023-02-27 15:50:44,315 | server.py:86 | Initializing global parameters
INFO flwr 2023-02-27 15:50:44,317 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-02-27 15:50:44,320 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-02-27 15:50:44,324 | server.py:101 | FL starting
DEBUG flwr 2023-02-27 15:50:44,327 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=16961) [Client 1] fit, config: {}
(launch_and_fit pid=16963) [Client 9] fit, config: {}
(launch_and_fit pid=16962) [Client 8] fit, config: {}
(launch_and_fit pid=16963) Epoch 1: train loss 0.06593215465545654, accuracy 0.22088888888888888
(launch_and_fit pid=16961) Epoch 1: train loss 0.06524761766195297, accuracy 0.2222222222222222


DEBUG flwr 2023-02-27 15:51:09,997 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-02-27 15:51:10,028 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-02-27 15:51:10,039 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=16962) Epoch 1: train loss 0.06476739794015884, accuracy 0.21288888888888888
(launch_and_evaluate pid=16961) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=16963) [Client 6] evaluate, config: {}


(raylet) E0227 15:51:28.906780400   17492 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509488.906747200","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:51:28.942513900   17507 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509488.942477300","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:51:30.526282900   17508 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509490.526099000","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","fil

...

(launch_and_evaluate pid=16959) [Client 1] evaluate, config: {}
(launch_and_fit pid=16961) [Client 7] fit, config: {}
(launch_and_fit pid=16959) [Client 0] fit, config: {}
(launch_and_fit pid=16963) [Client 1] fit, config: {}
(launch_and_fit pid=16961) Epoch 1: train loss 0.08753269910812378, accuracy 0.17644444444444443
(launch_and_fit pid=16959) Epoch 1: train loss 0.09109263867139816, accuracy 0.17177777777777778


DEBUG flwr 2023-02-27 15:52:24,003 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-02-27 15:52:24,021 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=16963) Epoch 1: train loss 0.08876537531614304, accuracy 0.184
(launch_and_evaluate pid=16963) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=16961) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=16959) [Client 1] evaluate, config: {}


(raylet) Spilled 2058 MiB, 25 objects, write throughput 118 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
DEBUG flwr 2023-02-27 15:52:30,545 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-02-27 15:52:30,546 | server.py:144 | FL finished in 106.2191026999999
INFO flwr 2023-02-27 15:52:30,547 | app.py:202 | app_fit: losses_distributed [(1, 6.097165802001953), (2, 0.43599525769551595), (3, 0.14549784088134768)]
INFO flwr 2023-02-27 15:52:30,549 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-02-27 15:52:30,550 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-02-27 15:52:30,551 | app.py:205 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 6.097165802001953
	round 2: 0.43599525769551595
	round 3: 0.14549784088134768

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [8]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [9]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-02-27 15:52:30,898 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-02-27 15:52:48,259	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-02-27 15:52:54,731 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 3021282510.0, 'node:10.246.68.42': 1.0, 'CPU': 8.0, 'object_store_memory': 1510641254.0}
INFO flwr 2023-02-27 15:52:54,742 | server.py:86 | Initializing global parameters
INFO flwr 2023-02-27 15:52:54,747 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-02-27 15:52:54,752 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-02-27 15:52:55,361 | server.py:91 | initial parameters (loss, other metrics): 0.07379200744628907, {'accuracy': 0.096}
INFO flwr 2023-02-27 15:52:55,364 | server.py:101 | FL starting
DEBUG flwr 2023-02-27 15:52:55,375 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 

Server-side evaluation loss 0.07379200744628907 / accuracy 0.096
(launch_and_fit pid=18133) [Client 8] fit, config: {}
(launch_and_fit pid=18134) [Client 0] fit, config: {}
(launch_and_fit pid=18135) [Client 2] fit, config: {}


DEBUG flwr 2023-02-27 15:53:12,852 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-02-27 15:53:12,862 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=18133) Epoch 1: train loss 0.06419975310564041, accuracy 0.23044444444444445
(launch_and_fit pid=18134) Epoch 1: train loss 0.06415960192680359, accuracy 0.24155555555555555
(launch_and_fit pid=18135) Epoch 1: train loss 0.06351277977228165, accuracy 0.23644444444444446


INFO flwr 2023-02-27 15:53:13,068 | server.py:116 | fit progress: (1, 0.0616036479473114, {'accuracy': 0.31}, 17.692429400000037)
DEBUG flwr 2023-02-27 15:53:13,069 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


...

(launch_and_fit pid=18134) Epoch 1: train loss 0.05138608440756798, accuracy 0.38755555555555554
(launch_and_fit pid=18133) Epoch 1: train loss 0.05228734388947487, accuracy 0.39844444444444443


INFO flwr 2023-02-27 15:53:50,612 | server.py:116 | fit progress: (3, 0.05148048067092895, {'accuracy': 0.428}, 55.23696760000075)
DEBUG flwr 2023-02-27 15:53:50,613 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=18130) Epoch 1: train loss 0.05163895711302757, accuracy 0.3933333333333333
Server-side evaluation loss 0.05148048067092895 / accuracy 0.428
(launch_and_evaluate pid=18130) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=18134) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=18132) [Client 1] evaluate, config: {}


DEBUG flwr 2023-02-27 15:54:01,253 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-02-27 15:54:01,255 | server.py:144 | FL finished in 65.87967420000041
INFO flwr 2023-02-27 15:54:01,256 | app.py:202 | app_fit: losses_distributed [(1, 0.06166497111320496), (2, 0.05276294088363647), (3, 0.05091049361228942)]
INFO flwr 2023-02-27 15:54:01,257 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-02-27 15:54:01,258 | app.py:204 | app_fit: losses_centralized [(0, 0.07379200744628907), (1, 0.0616036479473114), (2, 0.053530490636825565), (3, 0.05148048067092895)]
INFO flwr 2023-02-27 15:54:01,259 | app.py:205 | app_fit: metrics_centralized {'accuracy': [(0, 0.096), (1, 0.31), (2, 0.376), (3, 0.428)]}


History (loss, distributed):
	round 1: 0.06166497111320496
	round 2: 0.05276294088363647
	round 3: 0.05091049361228942
History (loss, centralized):
	round 0: 0.07379200744628907
	round 1: 0.0616036479473114
	round 2: 0.053530490636825565
	round 3: 0.05148048067092895
History (metrics, centralized):
{'accuracy': [(0, 0.096), (1, 0.31), (2, 0.376), (3, 0.428)]}

## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (trainig, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [10]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [11]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [12]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-02-27 15:54:01,832 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-02-27 15:54:12,711	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-02-27 15:54:17,699 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:10.246.68.42': 1.0, 'memory': 3294825678.0, 'object_store_memory': 1647412838.0, 'CPU': 8.0}
INFO flwr 2023-02-27 15:54:17,701 | server.py:86 | Initializing global parameters
INFO flwr 2023-02-27 15:54:17,702 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-02-27 15:54:17,703 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-02-27 15:54:17,948 | server.py:91 | initial parameters (loss, other metrics): 0.07383857822418213, {'accuracy': 0.102}
INFO flwr 2023-02-27 15:54:17,949 | server.py:101 | FL starting
DEBUG flwr 2023-02-27 15:54:17,950 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 

Server-side evaluation loss 0.07383857822418213 / accuracy 0.102
(launch_and_fit pid=18904) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=18908) [Client 6, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=18909) [Client 1, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-02-27 15:54:30,967 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-02-27 15:54:30,981 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=18904) Epoch 1: train loss 0.06377512961626053, accuracy 0.24488888888888888
(launch_and_fit pid=18908) Epoch 1: train loss 0.06503230333328247, accuracy 0.22088888888888888


INFO flwr 2023-02-27 15:54:31,216 | server.py:116 | fit progress: (1, 0.06176053500175476, {'accuracy': 0.304}, 13.26582099999996)
DEBUG flwr 2023-02-27 15:54:31,217 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


...

INFO flwr 2023-02-27 15:55:12,158 | server.py:116 | fit progress: (3, 0.04911686301231384, {'accuracy': 0.45}, 54.208160099999986)
DEBUG flwr 2023-02-27 15:55:12,159 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=18908) Epoch 2: train loss 0.04801885783672333, accuracy 0.44066666666666665
Server-side evaluation loss 0.04911686301231384 / accuracy 0.45
(launch_and_evaluate pid=18904) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=18908) [Client 8] evaluate, config: {}


DEBUG flwr 2023-02-27 15:55:20,202 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-02-27 15:55:20,203 | server.py:144 | FL finished in 62.2529013000003
INFO flwr 2023-02-27 15:55:20,204 | app.py:202 | app_fit: losses_distributed [(1, 0.06204264259338379), (2, 0.05329331096013387), (3, 0.04731542666753133)]
INFO flwr 2023-02-27 15:55:20,204 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-02-27 15:55:20,205 | app.py:204 | app_fit: losses_centralized [(0, 0.07383857822418213), (1, 0.06176053500175476), (2, 0.05338016629219055), (3, 0.04911686301231384)]
INFO flwr 2023-02-27 15:55:20,207 | app.py:205 | app_fit: metrics_centralized {'accuracy': [(0, 0.102), (1, 0.304), (2, 0.38), (3, 0.45)]}


(launch_and_evaluate pid=18908) [Client 7] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06204264259338379
	round 2: 0.05329331096013387
	round 3: 0.04731542666753133
History (loss, centralized):
	round 0: 0.07383857822418213
	round 1: 0.06176053500175476
	round 2: 0.05338016629219055
	round 3: 0.04911686301231384
History (metrics, centralized):
{'accuracy': [(0, 0.102), (1, 0.304), (2, 0.38), (3, 0.45)]}

As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [13]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [14]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-02-27 15:55:24,654 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-02-27 15:55:36,960	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-02-27 15:55:41,067 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1663985664.0, 'CPU': 8.0, 'memory': 3327971328.0, 'node:10.246.68.42': 1.0}
INFO flwr 2023-02-27 15:55:41,077 | server.py:86 | Initializing global parameters
INFO flwr 2023-02-27 15:55:41,078 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-02-27 15:55:41,080 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-02-27 15:55:41,081 | server.py:101 | FL starting
DEBUG flwr 2023-02-27 15:55:41,083 | server.py:215 | fit_round 1: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=19674) [Client 823, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=19674) Epoch 1: train loss 0.10284201055765152, accuracy 0.06666666666666667
(launch_and_fit pid=19674) Epoch 2: train loss 0.10172911733388901, accuracy 0.1111111111111111
(launch_and_fit pid=19674) Epoch 3: train loss 0.10133899748325348, accuracy 0.2


(raylet) E0227 15:56:06.904156700   20025 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509766.904120400","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:56:06.934880100   20026 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509766.934844000","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:56:06.966421600   20021 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509766.966383100","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","fil

(launch_and_fit pid=19675) [Client 724, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=19675) Epoch 1: train loss 0.10198123753070831, accuracy 0.08888888888888889
(launch_and_fit pid=19672) [Client 498, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=19673) [Client 962, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=19676) [Client 47, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=19675) Epoch 2: train loss 0.1017126739025116, accuracy 0.17777777777777778
(launch_and_fit pid=19675) Epoch 3: train loss 0.10045036673545837, accuracy 0.2222222222222222
(launch_and_fit pid=19672) Epoch 1: train loss 0.10291380435228348, accuracy 0.06666666666666667
(launch_and_fit pid=19673) Epoch 1: train loss 0.1025865226984024, accuracy 0.13333333333333333
(launch_and_fit pid=19676) Epoch 1: train loss 0.103032685816288, accuracy 0.06666666666666667
(launch_and_fit pid=

(raylet) E0227 15:56:19.929306200   20184 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509779.929271400","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:56:19.938908800   20182 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509779.938870700","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":202,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(raylet) E0227 15:56:19.965374500   20183 socket_utils_common_posix.cc:223] check for SO_REUSEPORT: {"created":"@1677509779.965342500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","fil

...

(launch_and_evaluate pid=21198) [Client 398] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 850] evaluate, config: {}
(launch_and_evaluate pid=20698) [Client 200] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 652] evaluate, config: {}
(launch_and_evaluate pid=21331) [Client 123] evaluate, config: {}
(launch_and_evaluate pid=21331) [Client 797] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 154] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 851] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 803] evaluate, config: {}
(launch_and_evaluate pid=21331) [Client 12] evaluate, config: {}
(launch_and_evaluate pid=21331) [Client 299] evaluate, config: {}
(launch_and_evaluate pid=20698) [Client 497] evaluate, config: {}
(launch_and_evaluate pid=21199) [Client 304] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 831] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 685] evaluate, config: {}
(launch_and

DEBUG flwr 2023-02-27 16:04:34,750 | server.py:179 | evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-02-27 16:04:34,770 | server.py:144 | FL finished in 533.6758999000003
INFO flwr 2023-02-27 16:04:34,772 | app.py:202 | app_fit: losses_distributed [(1, 0.4597410078048705), (2, 0.45852434730529773), (3, 0.4515949525833129)]
INFO flwr 2023-02-27 16:04:34,774 | app.py:203 | app_fit: metrics_distributed {}
INFO flwr 2023-02-27 16:04:34,776 | app.py:204 | app_fit: losses_centralized []
INFO flwr 2023-02-27 16:04:34,777 | app.py:205 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=21199) [Client 265] evaluate, config: {}
(launch_and_evaluate pid=21198) [Client 138] evaluate, config: {}


History (loss, distributed):
	round 1: 0.4597410078048705
	round 2: 0.45852434730529773
	round 3: 0.4515949525833129

(raylet) [2023-02-27 16:10:37,631 E 19577 19577] (raylet) node_manager.cc:3097: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 23822b6a08ac13c06b82c01db60956472938564608a9004253685752, IP: 10.246.68.42) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.246.68.42`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2023-02-27 16:11:37,638 E 19577 19577] (raylet) node_manager.cc:3097: 1 Workers (tasks / actors) killed due

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3 [WIP]](https://flower.dev/docs/tutorial/Flower-3-Building-a-Strategy-PyTorch.html) shows how to build a fully custom `Strategy` from scratch.